<img src="./ESSENCE.png" width=700px>

### Reflection pattern

We generate a response with our first query and then add this content to the request for a critique or further refinement.

The first request may be a standard request as in this case to generate some Python Code and the system prompt reflects this.

Then with the returned content, we create a second request with the content included, (RAG), with some instruction to critique it or refine it.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-y-


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a straightforward implementation of requesting data from an API using the `requests` library in Python. This example demonstrates how to make a GET request, handle potential errors, and parse the returned JSON data.

```python
import requests

def fetch_data_from_api(url, params=None):
    """
    Fetch data from the specified API endpoint.

    :param url: The URL of the API endpoint to request data from.
    :param params: Optional dictionary of query parameters to include in the request.
    :return: The JSON response data if the request is successful, otherwise None.
    """
    try:
        # Send a GET request to the API
        response = requests.get(url, params=params)

        # Raise an exception if the response status code is not 200 (OK)
        response.raise_for_status()

        # Parse and return the JSON response
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Error occurred during the request: {req_err}")
    except ValueError as json_err:
        print(f"Error parsing JSON response: {json_err}")

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_parameters = {'key': 'value'}  # Example query parameters
    data = fetch_data_from_api(api_url, query_parameters)

    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Explanation:
- The function `fetch_data_from_api` takes a URL and optional parameters.
- It uses the `requests.get` method to make the GET request.
- The function checks for HTTP errors and handles exceptions gracefully.
- If the request is successful, it returns the response parsed as JSON.
- If not, it prints an appropriate error message.
- An example usage is included in the `if __name__ == "__main__":` block, demonstrating how to call the function. 

Make sure to replace `https://api.example.com/data` and the query parameters with the actual API endpoint and parameters as required for your use case.

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your code demonstrates a solid approach for making an API request using the `requests` library in Python. Below are some critique points, along with recommendations for improvements and best practices that can enhance your implementation:

### Critique and Recommendations

1. **Parameter Validation**:
   - Consider validating the `url` parameter to ensure that it is a valid URL before attempting to make a request. This could prevent unnecessary requests and provide clearer error messages to users.

   ```python
   from urllib.parse import urlparse

   def is_valid_url(url):
       parsed = urlparse(url)
       return all([parsed.scheme, parsed.netloc])
   ```

   Before making the request, you might include:
   ```python
   if not is_valid_url(url):
       print("Invalid URL provided.")
       return None
   ```

2. **Logging Instead of Print**:
   - Instead of using `print` statements for error messages, consider using Python’s `logging` module. This allows for better control over logging levels and outputs, and it’s a more scalable solution for larger applications.

   ```python
   import logging

   logging.basicConfig(level=logging.INFO)
   ```

   Replace the print statements with logger calls:
   ```python
   logger.error("HTTP error occurred: %s", http_err)
   ```

3. **Return More Information**:
   - Instead of returning `None` on failure, consider returning a more informative object or error message. This can help the caller of the function determine what went wrong.

   ```python
   return {"error": str(req_err)}
   ```

4. **Timeouts**:
   - It's good practice to specify a timeout for your requests to avoid situations where your application hangs if the server does not respond. You can do this by adding a `timeout` parameter to the `requests.get()` call.

   ```python
   response = requests.get(url, params=params, timeout=10)
   ```

5. **Flexible Return Type**:
   - Depending on the intended use case, you might want to allow the function to return either the JSON response or raw text based on a parameter. This can be useful for APIs that return different content types.

   ```python
   def fetch_data_from_api(url, params=None, json_response=True):
       ...
       if json_response:
           return response.json()
       return response.text
   ```

6. **Docstring Enhancement**:
   - Your docstring provides a good overview, but you could include more information about the return type and what errors might be raised.

   ```python
   """
   Fetch data from the specified API endpoint.

   :param url: The URL of the API endpoint to request data from.
   :param params: Optional dictionary of query parameters to include in the request.
   :param json_response: Flag to indicate if the response should be parsed as JSON.
   :return: The JSON response data, raw text, or None if an error occurred.
   """
   ```

7. **Example Usage**:
   - In the example usage, ensure to test various scenarios, such as invalid URLs, non-200 status codes, and valid requests. This will help in verifying that your error handling works as expected.

Here’s an updated version of your function that incorporates some of these suggestions:

```python
import requests
import logging
from urllib.parse import urlparse

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    parsed = urlparse(url)
    return all([parsed.scheme, parsed.netloc])

def fetch_data_from_api(url, params=None, json_response=True):
    if not is_valid_url(url):
        logging.error("Invalid URL provided.")
        return None

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()

        if json_response:
            return response.json()
        return response.text

    except requests.exceptions.HTTPError as http_err:
        logging.error("HTTP error occurred: %s", http_err)
    except requests.exceptions.RequestException as req_err:
        logging.error("Error occurred during the request: %s", req_err)
    except ValueError as json_err:
        logging.error("Error parsing JSON response: %s", json_err)

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_parameters = {'key': 'value'}
    data = fetch_data_from_api(api_url, query_parameters)

    if data:
        logging.info("Data fetched successfully:")
        logging.info(data)
    else:
        logging.warning("Failed to fetch data.")
```

These recommendations should help improve the robustness, maintainability, and user-friendliness of your API fetching function.

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for the thoughtful critique and recommendations! Based on your feedback, I've integrated various improvements into the implementation of the API request function. Here's the revised version:

```python
import requests
import logging
from urllib.parse import urlparse

# Configure logging
logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """
    Validate the URL to ensure it is well-formed.

    :param url: The URL to validate.
    :return: True if the URL is valid, False otherwise.
    """
    parsed = urlparse(url)
    return all([parsed.scheme, parsed.netloc])

def fetch_data_from_api(url, params=None, json_response=True):
    """
    Fetch data from the specified API endpoint.

    :param url: The URL of the API endpoint to request data from.
    :param params: Optional dictionary of query parameters to include in the request.
    :param json_response: Flag to indicate if the response should be parsed as JSON.
    :return: The JSON response data, raw text, or None if an error occurred.
    """
    if not is_valid_url(url):
        logging.error("Invalid URL provided.")
        return None

    try:
        # Set a timeout for the request
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses

        # Return either JSON or raw text based on the flag
        return response.json() if json_response else response.text

    except requests.exceptions.HTTPError as http_err:
        logging.error("HTTP error occurred: %s", http_err)
    except requests.exceptions.RequestException as req_err:
        logging.error("General error occurred during the request: %s", req_err)
    except ValueError as json_err:
        logging.error("Error parsing JSON response: %s", json_err)

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    query_parameters = {'key': 'value'}  # Sample query parameters

    data = fetch_data_from_api(api_url, query_parameters)

    if data:
        logging.info("Data fetched successfully:")
        logging.info(data)
    else:
        logging.warning("Failed to fetch data.")
```

### Key Improvements:
1. **URL Validation**: A utility function `is_valid_url` has been added to check if the provided URL is valid, thus preventing unnecessary requests.

2. **Logging**: The usage of `logging` replaces `print` statements for better error reporting and control over log levels.

3. **Timeouts**: Included a timeout in the request to ensure that the application does not hang indefinitely.

4. **Flexible Response Handling**: The function can return either JSON or raw text based on the `json_response` parameter.

5. **Detailed Docstrings**: Each function now includes docstrings that clarify their purpose and parameters.

This revised implementation should address the concerns raised while enhancing the overall quality and usability of the API request function. Thank you for your feedback!

<br>

## We can of course make this a Class...
